# SGLang Native APIs

Apart from the OpenAI compatible APIs, the SGLang Runtime also provides its native server APIs. We introduce these following APIs:

- `/generate` (text generation model)
- `/get_model_info`
- `/get_server_info`
- `/health`
- `/health_generate`
- `/flush_cache`
- `/update_weights`
- `/encode`(embedding model)
- `/classify`(reward model)

We mainly use `requests` to test these APIs in the following examples. You can also use `curl`.

## Launch A Server

In [1]:
import requests
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path meta-llama/Llama-3.2-1B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")

[2025-03-12 07:19:16] server_args=ServerArgs(model_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.2-1B-Instruct', chat_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=30658, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, tp_size=1, stream_interval=1, stream_output=False, random_seed=922160098, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=False, log_requests_level=0, show_time_cost

[2025-03-12 07:19:35 TP0] Init torch distributed begin.
[2025-03-12 07:19:35 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-03-12 07:19:35 TP0] Load weight begin. avail mem=63.19 GB


[2025-03-12 07:19:36 TP0] The following error message 'operation scheduled before its operands' can be ignored.


[2025-03-12 07:19:36 TP0] Using model weights format ['*.safetensors']
[2025-03-12 07:19:36 TP0] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.02it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.02it/s]

[2025-03-12 07:19:37 TP0] Load weight end. type=LlamaForCausalLM, dtype=torch.bfloat16, avail mem=60.75 GB, mem usage=2.44 GB.
[2025-03-12 07:19:37 TP0] KV Cache is allocated. #tokens: 20480, K size: 0.31 GB, V size: 0.31 GB
[2025-03-12 07:19:37 TP0] Memory pool end. avail mem=59.97 GB


[2025-03-12 07:19:37 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072
[2025-03-12 07:19:37] INFO:     Started server process [3691307]
[2025-03-12 07:19:37] INFO:     Waiting for application startup.
[2025-03-12 07:19:37] INFO:     Application startup complete.
[2025-03-12 07:19:37] INFO:     Uvicorn running on http://0.0.0.0:30658 (Press CTRL+C to quit)


[2025-03-12 07:19:38] INFO:     127.0.0.1:36790 - "GET /v1/models HTTP/1.1" 200 OK


[2025-03-12 07:19:38] INFO:     127.0.0.1:36800 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-03-12 07:19:38 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-12 07:19:42] INFO:     127.0.0.1:36810 - "POST /generate HTTP/1.1" 200 OK
[2025-03-12 07:19:42] The server is fired up and ready to roll!


## Generate (text generation model)
Generate completions. This is similar to the `/v1/completions` in OpenAI API. Detailed parameters can be found in the [sampling parameters](./sampling_params.md).

In [2]:
url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-03-12 07:19:43 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-12 07:19:43 TP0] Decode batch. #running-req: 1, #token: 41, token usage: 0.00, gen throughput (token/s): 7.10, largest-len: 0, #queue-req: 0, 


[2025-03-12 07:19:43 TP0] Decode batch. #running-req: 1, #token: 81, token usage: 0.00, gen throughput (token/s): 165.94, largest-len: 0, #queue-req: 0, 


[2025-03-12 07:19:43 TP0] Decode batch. #running-req: 1, #token: 121, token usage: 0.01, gen throughput (token/s): 168.54, largest-len: 0, #queue-req: 0, 
[2025-03-12 07:19:44] INFO:     127.0.0.1:36824 - "POST /generate HTTP/1.1" 200 OK


## Get Model Info

Get the information of the model.

- `model_path`: The path/name of the model.
- `is_generation`: Whether the model is used as generation model or embedding model.
- `tokenizer_path`: The path/name of the tokenizer.

In [3]:
url = f"http://localhost:{port}/get_model_info"

response = requests.get(url)
response_json = response.json()
print_highlight(response_json)
assert response_json["model_path"] == "meta-llama/Llama-3.2-1B-Instruct"
assert response_json["is_generation"] is True
assert response_json["tokenizer_path"] == "meta-llama/Llama-3.2-1B-Instruct"
assert response_json.keys() == {"model_path", "is_generation", "tokenizer_path"}

[2025-03-12 07:19:44] INFO:     127.0.0.1:36828 - "GET /get_model_info HTTP/1.1" 200 OK


## Get Server Info
Gets the server information including CLI arguments, token limits, and memory pool sizes.
- Note: `get_server_info` merges the following deprecated endpoints:
  - `get_server_args`
  - `get_memory_pool_size` 
  - `get_max_total_num_tokens`

In [4]:
# get_server_info

url = f"http://localhost:{port}/get_server_info"

response = requests.get(url)
print_highlight(response.text)

[2025-03-12 07:19:44] INFO:     127.0.0.1:36834 - "GET /get_server_info HTTP/1.1" 200 OK


## Health Check
- `/health`: Check the health of the server.
- `/health_generate`: Check the health of the server by generating one token.

In [5]:
url = f"http://localhost:{port}/health_generate"

response = requests.get(url)
print_highlight(response.text)

[2025-03-12 07:19:44 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-12 07:19:45] INFO:     127.0.0.1:36840 - "GET /health_generate HTTP/1.1" 200 OK


In [6]:
url = f"http://localhost:{port}/health"

response = requests.get(url)
print_highlight(response.text)

[2025-03-12 07:19:45] INFO:     127.0.0.1:36846 - "GET /health HTTP/1.1" 200 OK


## Flush Cache

Flush the radix cache. It will be automatically triggered when the model weights are updated by the `/update_weights` API.

In [7]:
# flush cache

url = f"http://localhost:{port}/flush_cache"

response = requests.post(url)
print_highlight(response.text)

[2025-03-12 07:19:45] INFO:     127.0.0.1:36862 - "POST /flush_cache HTTP/1.1" 200 OK
[2025-03-12 07:19:45 TP0] Cache flushed successfully!


## Update Weights From Disk

Update model weights from disk without restarting the server. Only applicable for models with the same architecture and parameter size.

SGLang support `update_weights_from_disk` API for continuous evaluation during training (save checkpoint to disk and update weights from disk).


In [8]:
# successful update with same architecture and size

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "meta-llama/Llama-3.2-1B"}

response = requests.post(url, json=data)
print_highlight(response.text)
assert response.json()["success"] is True
assert response.json()["message"] == "Succeeded to update model weights."

[2025-03-12 07:19:45] Start update_weights. Load format=auto
[2025-03-12 07:19:45 TP0] Update engine weights online from disk begin. avail mem=74.84 GB


[2025-03-12 07:19:46 TP0] Using model weights format ['*.safetensors']
[2025-03-12 07:19:46 TP0] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.60it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.60it/s]

[2025-03-12 07:19:46 TP0] Update weights end.
[2025-03-12 07:19:46 TP0] Cache flushed successfully!
[2025-03-12 07:19:46] INFO:     127.0.0.1:36868 - "POST /update_weights_from_disk HTTP/1.1" 200 OK


In [9]:
# failed update with different parameter size or wrong name

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "meta-llama/Llama-3.2-1B-wrong"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(response_json)
assert response_json["success"] is False
assert response_json["message"] == (
    "Failed to get weights iterator: "
    "meta-llama/Llama-3.2-1B-wrong"
    " (repository not found)."
)

[2025-03-12 07:19:46] Start update_weights. Load format=auto
[2025-03-12 07:19:46 TP0] Update engine weights online from disk begin. avail mem=74.84 GB


[2025-03-12 07:19:47 TP0] Failed to get weights iterator: meta-llama/Llama-3.2-1B-wrong (repository not found).
[2025-03-12 07:19:47] INFO:     127.0.0.1:36880 - "POST /update_weights_from_disk HTTP/1.1" 400 Bad Request


## Encode (embedding model)

Encode text into embeddings. Note that this API is only available for [embedding models](openai_api_embeddings.html#openai-apis-embedding) and will raise an error for generation models.
Therefore, we launch a new server to server an embedding model.

In [10]:
terminate_process(server_process)

embedding_process, port = launch_server_cmd(
    """
python -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-7B-instruct \
    --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-03-12 07:20:00] server_args=ServerArgs(model_path='Alibaba-NLP/gte-Qwen2-7B-instruct', tokenizer_path='Alibaba-NLP/gte-Qwen2-7B-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Alibaba-NLP/gte-Qwen2-7B-instruct', chat_template=None, is_embedding=True, revision=None, host='0.0.0.0', port=33061, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, tp_size=1, stream_interval=1, stream_output=False, random_seed=461241059, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=False, log_requests_level=0, show_time_co

[2025-03-12 07:20:05] Downcasting torch.float32 to torch.float16.


[2025-03-12 07:20:19 TP0] Downcasting torch.float32 to torch.float16.


[2025-03-12 07:20:19 TP0] Overlap scheduler is disabled for embedding models.
[2025-03-12 07:20:19 TP0] Downcasting torch.float32 to torch.float16.
[2025-03-12 07:20:19 TP0] Init torch distributed begin.


[2025-03-12 07:20:19 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-03-12 07:20:19 TP0] Load weight begin. avail mem=78.81 GB
[2025-03-12 07:20:19 TP0] The following error message 'operation scheduled before its operands' can be ignored.


...

Loading safetensors checkpoint shards: 100% Completed | 7/7 [00:12<00:00,  1.91s/it]
Loading safetensors checkpoint shards: 100% Completed | 7/7 [00:12<00:00,  1.82s/it]

[2025-03-12 07:20:33 TP0] Load weight end. type=Qwen2ForCausalLM, dtype=torch.float16, avail mem=64.38 GB, mem usage=14.43 GB.
[2025-03-12 07:20:33 TP0] KV Cache is allocated. #tokens: 20480, K size: 0.55 GB, V size: 0.55 GB
[2025-03-12 07:20:33 TP0] Memory pool end. avail mem=63.07 GB


[2025-03-12 07:20:34 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072
[2025-03-12 07:20:34] INFO:     Started server process [3693626]
[2025-03-12 07:20:34] INFO:     Waiting for application startup.
[2025-03-12 07:20:34] INFO:     Application startup complete.
[2025-03-12 07:20:34] INFO:     Uvicorn running on http://0.0.0.0:33061 (Press CTRL+C to quit)


[2025-03-12 07:20:34] INFO:     127.0.0.1:49548 - "GET /v1/models HTTP/1.1" 200 OK


[2025-03-12 07:20:35] INFO:     127.0.0.1:49560 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-03-12 07:20:35 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-12 07:20:36] INFO:     127.0.0.1:49574 - "POST /encode HTTP/1.1" 200 OK
[2025-03-12 07:20:36] The server is fired up and ready to roll!


In [11]:
# successful encode for embedding model

url = f"http://localhost:{port}/encode"
data = {"model": "Alibaba-NLP/gte-Qwen2-7B-instruct", "text": "Once upon a time"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(f"Text embedding (first 10): {response_json['embedding'][:10]}")

[2025-03-12 07:20:39 TP0] Prefill batch. #new-seq: 1, #new-token: 4, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-03-12 07:20:39] INFO:     127.0.0.1:46440 - "POST /encode HTTP/1.1" 200 OK


In [12]:
terminate_process(embedding_process)

## Classify (reward model)

SGLang Runtime also supports reward models. Here we use a reward model to classify the quality of pairwise generations.

In [13]:
terminate_process(embedding_process)

# Note that SGLang now treats embedding models and reward models as the same type of models.
# This will be updated in the future.

reward_process, port = launch_server_cmd(
    """
python -m sglang.launch_server --model-path Skywork/Skywork-Reward-Llama-3.1-8B-v0.2 --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-03-12 07:20:52] server_args=ServerArgs(model_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', chat_template=None, is_embedding=True, revision=None, host='0.0.0.0', port=30127, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, tp_size=1, stream_interval=1, stream_output=False, random_seed=300029332, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=False, log_requests_

[2025-03-12 07:21:10 TP0] Overlap scheduler is disabled for embedding models.
[2025-03-12 07:21:10 TP0] Init torch distributed begin.


[2025-03-12 07:21:11 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-03-12 07:21:11 TP0] Load weight begin. avail mem=78.81 GB
[2025-03-12 07:21:11 TP0] The following error message 'operation scheduled before its operands' can be ignored.


[2025-03-12 07:21:11 TP0] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.21it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.30it/s]

[2025-03-12 07:21:15 TP0] Load weight end. type=LlamaForSequenceClassification, dtype=torch.bfloat16, avail mem=64.68 GB, mem usage=14.13 GB.
[2025-03-12 07:21:15 TP0] KV Cache is allocated. #tokens: 20480, K size: 1.25 GB, V size: 1.25 GB
[2025-03-12 07:21:15 TP0] Memory pool end. avail mem=61.96 GB


[2025-03-12 07:21:15 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072
[2025-03-12 07:21:15] INFO:     Started server process [3694569]
[2025-03-12 07:21:15] INFO:     Waiting for application startup.
[2025-03-12 07:21:15] INFO:     Application startup complete.
[2025-03-12 07:21:15] INFO:     Uvicorn running on http://0.0.0.0:30127 (Press CTRL+C to quit)


[2025-03-12 07:21:16] INFO:     127.0.0.1:50856 - "GET /v1/models HTTP/1.1" 200 OK


[2025-03-12 07:21:16] INFO:     127.0.0.1:50866 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-03-12 07:21:16 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-12 07:21:17] INFO:     127.0.0.1:50876 - "POST /encode HTTP/1.1" 200 OK
[2025-03-12 07:21:17] The server is fired up and ready to roll!


In [14]:
from transformers import AutoTokenizer

PROMPT = (
    "What is the range of the numeric output of a sigmoid node in a neural network?"
)

RESPONSE1 = "The output of a sigmoid node is bounded between -1 and 1."
RESPONSE2 = "The output of a sigmoid node is bounded between 0 and 1."

CONVS = [
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE1}],
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE2}],
]

tokenizer = AutoTokenizer.from_pretrained("Skywork/Skywork-Reward-Llama-3.1-8B-v0.2")
prompts = tokenizer.apply_chat_template(CONVS, tokenize=False)

url = f"http://localhost:{port}/classify"
data = {"model": "Skywork/Skywork-Reward-Llama-3.1-8B-v0.2", "text": prompts}

responses = requests.post(url, json=data).json()
for response in responses:
    print_highlight(f"reward: {response['embedding'][0]}")

[2025-03-12 07:21:26 TP0] Prefill batch. #new-seq: 1, #new-token: 68, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-03-12 07:21:27 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 62, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-03-12 07:21:27] INFO:     127.0.0.1:41988 - "POST /classify HTTP/1.1" 200 OK


In [15]:
terminate_process(reward_process)

## Skip Tokenizer and Detokenizer

SGLang Runtime also supports skip tokenizer and detokenizer. This is useful in cases like integrating with RLHF workflow.

In [16]:
tokenizer_free_server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path meta-llama/Llama-3.2-1B-Instruct --skip-tokenizer-init
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-03-12 07:21:40] server_args=ServerArgs(model_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=True, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.2-1B-Instruct', chat_template=None, is_embedding=False, revision=None, host='127.0.0.1', port=30820, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, tp_size=1, stream_interval=1, stream_output=False, random_seed=367757339, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=False, log_requests_level=0, show_time_cos

[2025-03-12 07:21:59 TP0] Init torch distributed begin.
[2025-03-12 07:21:59 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-03-12 07:21:59 TP0] Load weight begin. avail mem=78.81 GB


[2025-03-12 07:21:59 TP0] The following error message 'operation scheduled before its operands' can be ignored.


[2025-03-12 07:21:59 TP0] Using model weights format ['*.safetensors']


[2025-03-12 07:22:00 TP0] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.09it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.09it/s]

[2025-03-12 07:22:01 TP0] Load weight end. type=LlamaForCausalLM, dtype=torch.bfloat16, avail mem=76.37 GB, mem usage=2.44 GB.
[2025-03-12 07:22:01 TP0] KV Cache is allocated. #tokens: 20480, K size: 0.31 GB, V size: 0.31 GB
[2025-03-12 07:22:01 TP0] Memory pool end. avail mem=75.58 GB


[2025-03-12 07:22:01 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072
[2025-03-12 07:22:01] INFO:     Started server process [3695513]
[2025-03-12 07:22:01] INFO:     Waiting for application startup.
[2025-03-12 07:22:01] INFO:     Application startup complete.
[2025-03-12 07:22:01] INFO:     Uvicorn running on http://127.0.0.1:30820 (Press CTRL+C to quit)


[2025-03-12 07:22:02] INFO:     127.0.0.1:51266 - "GET /v1/models HTTP/1.1" 200 OK


[2025-03-12 07:22:02] INFO:     127.0.0.1:51268 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-03-12 07:22:02 TP0] Prefill batch. #new-seq: 1, #new-token: 3, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-12 07:22:06] INFO:     127.0.0.1:51272 - "POST /generate HTTP/1.1" 200 OK
[2025-03-12 07:22:06] The server is fired up and ready to roll!


In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

input_text = "What is the capital of France?"

input_tokens = tokenizer.encode(input_text)
print_highlight(f"Input Text: {input_text}")
print_highlight(f"Tokenized Input: {input_tokens}")

response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "input_ids": input_tokens,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 256,
            "stop_token_ids": [tokenizer.eos_token_id],
        },
        "stream": False,
    },
)
output = response.json()
output_tokens = output["output_ids"]

output_text = tokenizer.decode(output_tokens, skip_special_tokens=False)
print_highlight(f"Tokenized Output: {output_tokens}")
print_highlight(f"Decoded Output: {output_text}")
print_highlight(f"Output Text: {output['meta_info']['finish_reason']}")

[2025-03-12 07:22:07 TP0] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-03-12 07:22:07 TP0] Decode batch. #running-req: 1, #token: 41, token usage: 0.00, gen throughput (token/s): 6.07, largest-len: 0, #queue-req: 0, 


[2025-03-12 07:22:08 TP0] Decode batch. #running-req: 1, #token: 81, token usage: 0.00, gen throughput (token/s): 196.73, largest-len: 0, #queue-req: 0, 
[2025-03-12 07:22:08] INFO:     127.0.0.1:51280 - "POST /generate HTTP/1.1" 200 OK


In [18]:
terminate_process(tokenizer_free_server_process)

[2025-03-12 07:22:08] Child process unexpectedly failed with an exit code 9. pid=3695652
